In [1]:
import os
import shutil
from time import sleep
from pprint import pprint

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
import numpy as np

from batchflow.models.tf import ScalableModel

from batchflow import Pipeline, B, V, C, D
from batchflow.opensets import MNIST
from batchflow.research import Option, Research, Results

W0911 15:56:27.842364 139755715958592 deprecation_wrapper.py:119] From /media/data/Data/source/github/analysiscenter/batchflow/batchflow/models/tf/losses/dice.py:6: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0911 15:56:27.844434 139755715958592 deprecation_wrapper.py:119] From /media/data/Data/source/github/analysiscenter/batchflow/batchflow/models/tf/losses/dice.py:6: The name tf.losses.Reduction is deprecated. Please use tf.compat.v1.losses.Reduction instead.

W0911 15:56:27.850976 139755715958592 deprecation_wrapper.py:119] From /media/data/Data/source/github/analysiscenter/batchflow/batchflow/models/tf/base.py:22: The name tf.losses.mean_squared_error is deprecated. Please use tf.compat.v1.losses.mean_squared_error instead.

W0911 15:56:27.852304 139755715958592 deprecation_wrapper.py:119] From /media/data/Data/source/github/analysiscenter/batchflow/batchflow/models/tf/base.py:23: The name tf.losses.sigmoid_cross_entropy is deprecated. Please 

In [2]:
dataset = MNIST()

BATCH_SIZE = 64

In [3]:
model_config = {
    'inputs/images/shape': B('image_shape'),
    'inputs/labels/classes': D('num_classes'),
    'initial_block/inputs': 'images',
    
    'initial_block': dict(layout='cna cna', kernel_size=3, filters=8),
    
    'common/width_factor': C('width_factor'),
    'common/depth_factor': C('depth_factor'),
    
    'loss': 'crossentropy',
    'optimizer': 'Adam', 
}

train_ppl = (dataset.train.p
                  .init_variable('loss') 
                  .init_model('dynamic', ScalableModel, 'mnist_model', config=model_config)
                  .to_array()
                  .train_model('mnist_model', fetches='loss',
                               images=B('images'), labels=B('labels'), 
                               save_to=V('loss'))
             .run_later(BATCH_SIZE, shuffle=True, n_epochs=None))

test_ppl = (dataset.test.p
                 .import_model('mnist_model', C('import_from'))
                 .init_variable('loss')
                 .init_variable('predictions')
                 .init_variable('metrics')
                 .to_array()
                 .predict_model('mnist_model', fetches=['loss', 'predictions'],
                                images=B('images'), labels=B('labels'), 
                                save_to=[V('loss'), V('predictions')])
                 .gather_metrics('class', targets=B('labels'), predictions=V('predictions'),
                                 fmt='logits', axis=-1, save_to=V('metrics'))
            .run_later(BATCH_SIZE, shuffle=True, n_epochs=1))


In [4]:
op1 = Option('width_factor', [1, 2])
op2 = Option('depth_factor', [1, 2])
grid = op1 * op2

list(grid.gen_configs())

[ConfigAlias({'width_factor': '1', 'depth_factor': '1'}),
 ConfigAlias({'width_factor': '1', 'depth_factor': '2'}),
 ConfigAlias({'width_factor': '2', 'depth_factor': '1'}),
 ConfigAlias({'width_factor': '2', 'depth_factor': '2'})]

In [5]:
research = (Research()
            .add_pipeline(train_ppl, variables='loss', name='train')
            .add_pipeline(test_ppl, variables='loss', name='test', execute=10,
                          run=True, import_from='train')
            .get_metrics(pipeline='test', metrics_var='metrics', metrics_name='accuracy',
                         returns='accuracy', execute=10)
            .add_grid(grid))

In [6]:
!rm -r scalable_research

In [ ]:
NUM_REPEATS = 3
NUM_ITERS = 200
research_name = 'scalable_research'

research.run(n_reps=NUM_REPEATS, n_iters=NUM_ITERS, name=research_name, bar=True)


Research scalable_research is starting...


  0%|          | 0/2400 [00:00<?, ?it/s]

Distributor has 12 jobs with 200 iterations. Totally: 2400


W0911 15:57:11.939679 139755517884160 deprecation_wrapper.py:119] From /media/data/Data/source/github/analysiscenter/batchflow/batchflow/models/tf/base.py:319: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0911 15:57:11.955456 139755517884160 deprecation_wrapper.py:119] From /media/data/Data/source/github/analysiscenter/batchflow/batchflow/models/tf/base.py:321: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0911 15:57:11.966859 139755517884160 deprecation_wrapper.py:119] From /media/data/Data/source/github/analysiscenter/batchflow/batchflow/models/tf/base.py:380: The name tf.get_variable_scope is deprecated. Please use tf.compat.v1.get_variable_scope instead.

W0911 15:57:11.973403 139755517884160 deprecation.py:323] From /media/data/Data/source/github/analysiscenter/batchflow/batchflow/models/tf/layers/conv.py:26: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed 

W0911 16:00:18.973582 139755517884160 deprecation.py:323] From /media/data/Data/source/github/analysiscenter/batchflow/batchflow/models/tf/layers/block.py:221: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0911 16:00:19.239926 139755517884160 deprecation_wrapper.py:119] From /media/data/Data/source/github/analysiscenter/batchflow/batchflow/models/tf/base.py:347: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0911 16:00:19.241936 139755517884160 deprecation_wrapper.py:119] From /media/data/Data/source/github/analysiscenter/batchflow/batchflow/models/tf/base.py:764: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

W0911 16:00:19.286464 139755517884160 deprecation.py:323] From /home/antonina/miniconda3/lib/python3.7/site-packages/tensorflow/python/ops/losses/losses_impl.py:121: add_dispatch_suppor

W0911 16:06:11.234580 139755517884160 deprecation_wrapper.py:119] From /media/data/Data/source/github/analysiscenter/batchflow/batchflow/models/tf/base.py:380: The name tf.get_variable_scope is deprecated. Please use tf.compat.v1.get_variable_scope instead.

W0911 16:06:11.240755 139755517884160 deprecation.py:323] From /media/data/Data/source/github/analysiscenter/batchflow/batchflow/models/tf/layers/conv.py:26: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0911 16:06:11.245244 139755517884160 deprecation.py:506] From /home/antonina/miniconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructo

W0911 16:10:05.908171 139755517884160 deprecation_wrapper.py:119] From /media/data/Data/source/github/analysiscenter/batchflow/batchflow/models/tf/base.py:347: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0911 16:10:05.912107 139755517884160 deprecation_wrapper.py:119] From /media/data/Data/source/github/analysiscenter/batchflow/batchflow/models/tf/base.py:764: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.

W0911 16:10:05.962104 139755517884160 deprecation.py:323] From /home/antonina/miniconda3/lib/python3.7/site-packages/tensorflow/python/ops/losses/losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
 66%|██████▌   | 1580/2400 [15:23<07:59,  1.71it/s]

In [ ]:
res = research.load_results(use_alias=True)
res.info()

In [ ]:
from batchflow.utils import plot_results_by_config

plot_results_by_config(res, (('train', 'loss'), ('test', 'loss'), ('test_metrics', 'accuracy')))